In [1]:
import numpy as np
import cv2
import pandas as pd
import functools
import os

In [ ]:
help(cv2.calibrateCamera)

In [2]:
def draw_line (img, rho, theta, color=(0,255,0)):
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(np.around(x0 + 1000*(-b)))   # Here i have used int() instead of rounding the decimal value, so 3.8 --> 3
    y1 = int(np.around(y0 + 1000*(a)))    # But if you want to round the number, then use np.around() function, then 3.8 --> 4.0
    x2 = int(np.around(x0 - 1000*(-b)))   # But we need integers, so use int() function after that, ie int(np.around(x))
    y2 = int(np.around(y0 - 1000*(a)))
    cv2.line(img,(x1,y1),(x2,y2),color,2)
    #print (rho, theta, color)

# Given y, this function returns x from (rho, theta) line equation
def x_from_y(rho, theta, y):
    sin_t = np.sin(theta)
    cos_t = np.cos(theta) 

    return ((rho / cos_t) - ((y * sin_t) / cos_t))

# Given x, this function returns y from (rho, theta) line equation
def y_from_x(rho, theta, x):
    sin_t = np.sin(theta)
    cos_t = np.cos(theta) 

    return (-((x * cos_t) / sin_t) + (rho / sin_t)) 

# Given a point and the rectangle, this function checks 
# if this point lies inside the rectangle
def check_point(x, y, left_x, right_x, up_y, down_y):
    if (left_x <= x) and (x <= right_x) and (up_y <= y) and (y <= down_y):
        return (int(np.around(x)), int(np.around(y)))
    else:
        return None

# A line which traverses some rectangle has two points of traversal.
#
# This function finds four possible traversal points and drops those of them 
# which are not inside the rectangle
def get_traversal_points(rho, theta, left_x, right_x, up_y, down_y):      

    #Find four possible points of traversal
    up = (x_from_y(rho, theta, up_y), up_y)
    right = (right_x, y_from_x(rho, theta, right_x))
    down = (x_from_y(rho, theta, down_y), down_y)
    left = (left_x, y_from_x(rho, theta, left_x))

    #If point is inside, we add it to the list; otherwise we add None
    intersect_points = list(map(lambda p: check_point(p[0], p[1], left_x, right_x, up_y, down_y), [up, right, down, left]))

    #Drop all None values
    #intersect_points = [point for point in intersect_points if point != None]
    intersect_points = [down, up]

    #Check if exactly two points were found
    if len(intersect_points) != 2:
        print('Error! len(intersect_points) != 2, intersect_points = {}'.format(intersect_points))
        return None
    else:
        return intersect_points


# Get the point of traversal for 'plus' and 'minus' lines
# The order as follows in a way to obtain convenient trapezoid
def find_intersection_point(rho_1, theta_1, rho_2, theta_2):
    k_1 = - np.cos(theta_1) / np.sin(theta_1)
    m_1 = rho_1 / np.sin(theta_1)

    k_2 = - np.cos(theta_2) / np.sin(theta_2)
    m_2 = rho_2 / np.sin(theta_2)

    x = (m_1 - m_2) / (k_1 - k_2)
    y = (k_1 * m_2 - k_2 * m_1) / (k_1 - k_2)

    return (x, y)

In [3]:
def preprocess_image(im, output_folder, filename):
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    equ = cv2.equalizeHist(gray)
    winSize = 3
    blured = cv2.GaussianBlur(equ, (winSize, winSize), 0)
    edges = cv2.Canny(blured,100,220,apertureSize = 3, L2gradient=True)

    #cv2.imwrite(output_folder+'gray_'+filename, gray)
    #cv2.imwrite(output_folder+'equ_'+filename, equ)
    #cv2.imwrite(output_folder+'blured_'+filename, blured)
    #cv2.imwrite(output_folder+'edges_'+filename, edges)
    
    return edges

In [ ]:
def find_horizontal_infinity_point (im, base):
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    equ = cv2.equalizeHist(gray)
    #ret, thresh = cv2.threshold(equ,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    winSize = 7
    blured = cv2.GaussianBlur(equ, (winSize, winSize), 0)
    edges = cv2.Canny(blured,100,220,apertureSize = 3, L2gradient=True)

    cv2.imwrite(base+'gray.jpg', gray)
    cv2.imwrite(base+'equ.jpg', equ)
    cv2.imwrite(base+'blured.jpg', blured)
    cv2.imwrite(base+'edges.jpg', edges)

    #cv2.imwrite(base+'thresh.jpg', thresh)
    print(im.shape)

    height = im.shape[0]
    width = im.shape[1]

    img = im.copy()
    rho_minus_min = -1100000
    rho_minus_max = 0
    rho_plus_min = 100000
    rho_plus_max = 0
    
    theta_minus_min = 0
    theta_minus_max = 0
    theta_plus_min = 0
    theta_plus_max = 0

    
    eps = 0.001
    lines = cv2.HoughLines(edges,1,np.pi/180, int (min(width,height)/6.))
    #print (lines)
    for line in lines:
        for rho,theta in line:
            if (theta <=  np.pi/2 + eps and np.sin(theta) > width/np.sqrt(height**2+width**2)):
                if (rho > rho_plus_max):
                    rho_plus_max = rho
                    theta_plus_max = theta
                if (rho < rho_plus_min):
                    rho_plus_min = rho
                    theta_plus_min = theta
                #draw_line (img, rho, theta)
            if (theta > np.pi/2 + eps and np.sin(theta) > width/np.sqrt(height**2+width**2)):
                if (rho > rho_minus_min):
                    rho_minus_min = rho
                    theta_minus_min = theta
                if (rho < rho_minus_max):
                    rho_minus_max = rho
                    theta_minus_max = theta
                #draw_line (img, rho, theta)

    red = (0, 0, 255)
    blue = (255, 0, 0)
    print (rho_minus_min, rho_plus_min, rho_minus_max, rho_plus_max)
    print (theta_minus_min, theta_plus_min, theta_minus_max, theta_plus_max)
    print (np.sin([theta_minus_min, theta_plus_min, theta_minus_max, theta_plus_max]))
    draw_line (img, rho_minus_min, theta_minus_min, (0, 0, 255))
    draw_line (img, rho_plus_min, theta_plus_min, (255, 0, 0))
    draw_line (img, rho_minus_max, theta_minus_max, (0, 0, 255))
    draw_line (img, rho_plus_max, theta_plus_max, (255, 0, 0))

    cv2.imwrite(base+'houghlines_hor.jpg', img)
    # Get the point of traversal for 'plus' and 'minus' lines
    # The order as follows in a way to obtain convenient trapezoid
    
    van_minus = (get_vanishing_point_safe (rho_minus_min, theta_minus_min, rho_minus_max, theta_minus_max))
    van_plus = (get_vanishing_point_safe (rho_plus_min, theta_plus_min, rho_plus_max, theta_plus_max))
    print (van_plus, van_minus)
    if (is_inside_image(im, van_plus)):
        return van_minus
    if (is_inside_image(im, van_minus)):
        return van_plus
    
    if not is_infty(van_plus):
        return van_plus
    if not is_infty(van_minus):
        return van_minus
    return van_plus
    

def is_infty (point):
    x,y = point
    return x == np.infty or y == np.infty

def is_inside_image (img, point):
    x,y = point
    width = im.shape[1]
    height = im.shape[0]
    if (x == np.infty or y == np.infty):
        return False
    x = int(x)
    y = int(y)
    return x >= 0 and x < width and y >= 0 and y < height

def get_vanishing_point_safe(rho_1, theta_1, rho_2, theta_2):
    rho_1 = np.abs (rho_1)
    rho_2 = np.abs (rho_2)
    if (np.abs (theta_1 - theta_2) < 0.01):
        return (np.infty, np.infty)
    m1 = rho_2 * np.sin(theta_1) - rho_1 * np.sin(theta_2)
    m2 = np.cos(theta_1) * np.sin(theta_2) - np.cos(theta_2) * np.sin(theta_1)
    van_x = m1 / m2
    if (np.abs(theta_1 - np.pi/2) > 0.01):
        van_y = y_from_x(rho_1, theta_1, van_x)
    else:
        van_y = y_from_x(rho_2, theta_2, van_x)
    return van_x, van_y

In [34]:
def find_vanishing_point(im, edges, output_folder, filename, line_tresh):
    img = im.copy()
    rho_minus = 0
    rho_plus = np.inf

    eps = 0.01
    lines = cv2.HoughLines(edges, 1, np.pi/180 , int (min(im.shape[0], im.shape[1]) / line_tresh))
    
    red = (0, 0, 255)
    
    for line in lines:
        for rho,theta in line:
            draw_line (img, rho, theta)
            if (theta > np.pi / 2 + eps and np.abs (rho) > np.abs (rho_minus)):
                rho_minus = rho
                theta_minus = theta
            if (theta < np.pi / 2 - eps and rho < rho_plus):
                rho_plus = rho
                theta_plus = theta

    
    draw_line (img, rho_minus, theta_minus, red)
    draw_line (img, rho_plus, theta_plus, red)

    #cv2.imwrite(output_folder+'houghlines_red_'+filename, img)


    van_x, van_y = find_intersection_point(rho_plus, theta_plus, rho_minus, theta_minus)
    
    return van_x, van_y

In [15]:
def find_perspective_transform(im, van_y):
    height = im.shape[0]
    width = im.shape[1]
    
    f = np.sqrt(width**2+height**2)
    van_dist = abs(- height/ 2 + van_y)
    print ('f = {}, van_dist = {}'.format(f, van_dist))
    
    theta = np.arctan(van_dist / f)
    cos_t = np.cos(theta)
    sin_t = np.sin(theta)
    tg_t = np.tan(theta)

    raw_from_points = [(0, 0), (0, f / tg_t), (f / sin_t, f / tg_t), (f / (2 * sin_t), f / np.tan(2 * theta))]
    raw_to_points = [(0, -f / tg_t), (0, 0), (f, 0), (f, -f * tg_t)]

    from_points = np.float32([(x + (width / 2), y + (height / 2)) for x, y in raw_from_points])
    to_points = np.float32([(x + (width / 2), y + (height / 2)) for x, y in raw_to_points])

    M = cv2.getPerspectiveTransform(from_points, to_points)
    
    return M, from_points, to_points    

In [14]:
def find_shifted_perspective_transform(im, van_y, from_points, to_points, M):
    height = im.shape[0]
    width = im.shape[1]
    
    if (van_y > 0):
        offset = 75
        input_corners = np.array([[(0, height), (0, van_y + offset), (width, van_y + offset), (width, height)]], dtype=float)
    else:
        print('Normal case')
        input_corners = np.array([[(0, height), (0,  0), (width,  0), (width, height)]], dtype=float)
    output_corners = cv2.perspectiveTransform(input_corners, M)
    print('output_corners = {}'.format(output_corners))


    x_min = functools.reduce(lambda x,y: [min(x[0], y[0]), 0], output_corners[0])[0]
    x_max = functools.reduce(lambda x,y: [max(x[0], y[0]), 0], output_corners[0])[0]
    y_min = functools.reduce(lambda x,y: [0, min(x[1], y[1])], output_corners[0])[1]
    y_max = functools.reduce(lambda x,y: [0, max(x[1], y[1])], output_corners[0])[1]

    br_x = x_min
    br_y = y_min

    new_width = int(x_max - x_min)
    new_height = int(y_max - y_min)

    print('new_width = {}, new_height = {}'.format(new_width, new_height))
    to_points = np.float32([(to_point[0] - br_x, to_point[1] - br_y) for to_point in to_points])
    M_shifted = cv2.getPerspectiveTransform(from_points, to_points)
    
    return M_shifted, new_width, new_height

In [26]:
test_points = 0
def CorrectPerspectiveDistortion (input_folder, filename, output_folder, line_thresh):
    #Read the image
    base = input_folder
    im = cv2.imread(base + filename)
    
    #Preprocess image: to gray, blur and find edges
    edges = preprocess_image(im, output_folder, filename)

    van_x, van_y = find_vanishing_point(im, edges, output_folder, filename, line_thresh)
    print('van_y = {}'.format(van_y))    
    
    M, from_points, to_points = find_perspective_transform(im, van_y)
    
    M_shifted, new_width, new_height = find_shifted_perspective_transform(im, van_y, from_points, to_points, M)
   
    dst = cv2.warpPerspective(im, M_shifted, (new_width, new_height))
    cv2.imwrite(output_folder + 'Transformed_{}.jpg'.format(filename), dst)
    
    return dst

In [35]:
test_points = 0
for file in os.listdir('test_images'):
    print(file)
    line_thresh = 2.5
    if file == '4.jpg':
        line_thresh = 2.6
    img = CorrectPerspectiveDistortion ('test_images/', file, 'output_images/', line_thresh)
    print()

4.jpg
van_y = -107.23995208740234
f = 1081.6653826391969, van_dist = 557.2399520874023
Normal case
output_corners = [[[   -62.40572123   -462.6347988 ]
  [ -3103.85298004 -12798.38786202]
  [  3703.84769813 -12798.36695447]
  [   662.40565078   -462.63464546]]]
new_width = 6807, new_height = 12335

5.png
van_y = -297.1356201171875
f = 931.6104336040897, van_dist = 675.6356201171875
Normal case
output_corners = [[[  -24.90117657  -202.2316637 ]
  [ -780.02997858 -3403.02593664]
  [ 1323.02996496 -3403.02581368]
  [  567.90116806  -202.23165838]]]
new_width = 2103, new_height = 3200

2.jpg
van_y = -712.34521484375
f = 1280.0, van_dist = 1224.34521484375
Normal case
output_corners = [[[   -7.72524682   -70.56575447]
  [ -570.83241754 -2668.00992695]
  [ 1338.83240064 -2668.00992656]
  [  775.72525369   -70.56575444]]]
new_width = 1909, new_height = 2597

3.jpg
van_y = -1681.44287109375
f = 510.4164965986111, van_dist = 1896.44287109375
Normal case
output_corners = [[[   9.60623548  284.72

## Original code is below

In [ ]:
# help(cv2.threshold)

#Read the image
base = './test_images/'
im = cv2.imread(base + '2.jpg')
gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
equ = cv2.equalizeHist(gray)
#ret, thresh = cv2.threshold(equ,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
winSize = 3
blured = cv2.GaussianBlur(equ, (winSize, winSize), 0)
edges = cv2.Canny(blured,100,220,apertureSize = 3, L2gradient=True)



#cv2.imshow('houghlines',im)
cv2.imwrite(base+'gray.jpg', gray)
cv2.imwrite(base+'equ.jpg', equ)
cv2.imwrite(base+'blured.jpg', blured)
cv2.imwrite(base+'edges.jpg', edges)

#cv2.imwrite(base+'thresh.jpg', thresh)

#A few changes to check git functionality
print(im.shape)

height = im.shape[0]
width = im.shape[1]


img = im.copy()
rho_minus = 0
rho_plus = np.inf

rho_hor_max = 0
hor_max_found = False
rho_hor_min = im.shape[0]
hor_min_found = False
eps = 0.01
lines = cv2.HoughLines(edges,1,np.pi/180, int (min(im.shape[0], im.shape[1]) / 3))
#print (lines)
for line in lines:
    for rho,theta in line:
        if (theta > np.pi / 2 + eps and np.abs (rho) > np.abs (rho_minus)):
            rho_minus = rho
            theta_minus = theta
        if (theta < np.pi / 2 - eps and rho < rho_plus):
            rho_plus = rho
            theta_plus = theta
        if (np.abs (theta - np.pi / 2) < eps):
            if (rho > rho_hor_max):
                rho_hor_max = rho
                hor_max_found = True
            if (rho < rho_hor_min):
                rho_hor_min = rho
                hor_min_found = True
        #draw_line (img, rho, theta)
        
red = (0, 0, 255)
draw_line (img, rho_minus, theta_minus, red)
draw_line (img, rho_plus, theta_plus, red)

cv2.imwrite(base+'houghlines_red.jpg', img)


# Get the point of traversal for 'plus' and 'minus' lines
# The order as follows in a way to obtain convenient trapezoid

van_x, van_y = get_vanishing_point(rho_plus, theta_plus, rho_minus, theta_minus)




In [ ]:
height

In [ ]:
f = np.sqrt(width**2+height**2) * 2.
print (van_y)
is_dist = height/ 2 - van_y
print (f, is_dist)
sin_alpha = f / np.sqrt(f**2 + is_dist**2)
cos_alpha = is_dist / np.sqrt(f**2 + is_dist**2)
tilt_matrix = np.array([[1, 0, 0],
                        [0, cos_alpha, -sin_alpha],
                        [0, sin_alpha, cos_alpha]])
shift_matrix = np.array([[1, 0, -width / 2 / f],
                         [0, 1, 0],
                         [0, 0, 1]])
M = np.dot(tilt_matrix, shift_matrix)
print (plus_down, plus_up)
start = [
    [-width/2, height, f],
    [-width/2, 0, f],
    #[plus_up[0], plus_up[1], f],
    #[plus_down[0], plus_down[1], f],
    [width/2, 0, f],
    [width/2, height, f]
]
print (start)
t_st = np.float32([(x[0],x[1]) for x in start])
finish = []
for x in start:
    tmp = np.dot(M, np.transpose(x))
    finish.append ((tmp[0]*f/tmp[2], tmp[1]*f/tmp[2]))
print (finish)

matr = cv2.getPerspectiveTransform(t_st, np.array([finish], dtype=np.float32))

x_min = functools.reduce(lambda x,y: [min(x[0], y[0]), 0], finish)[0]
x_max = functools.reduce(lambda x,y: [max(x[0], y[0]), 0], finish)[0]
y_min = functools.reduce(lambda x,y: [0, min(x[1], y[1])], finish)[1]
y_max = functools.reduce(lambda x,y: [0, max(x[1], y[1])], finish)[1]

br_x = x_min / 2
br_y = y_min

new_width = int(x_max - x_min)
new_height = int(y_max - y_min)

to_points = np.float32([(to_point[0] - br_x, to_point[1] - br_y) for to_point in finish])
M_shifted = cv2.getPerspectiveTransform(t_st, to_points)

dst = cv2.warpPerspective(im, M_shifted, (new_width, new_height))
cv2.imwrite(base+'transformed1.jpg', dst)

# My point of view

In [ ]:
f = np.sqrt(width**2+height**2)
print (van_y)
van_dist = abs(- height/ 2 + van_y)
print (f, van_dist)

theta = np.arctan(van_dist / f)
cos_t = np.cos(theta)
sin_t = np.sin(theta)
tg_t = np.tan(theta)

raw_from_points = [(0, 0), (0, f / tg_t), (f / sin_t, f / tg_t), (f / (2 * sin_t), f / np.tan(2 * theta))]
raw_to_points = [(0, -f / tg_t), (0, 0), (f, 0), (f, -f * tg_t)]

from_points = np.float32([(x + (width / 2), y + (height / 2)) for x, y in raw_from_points])
to_points = np.float32([(x + (width / 2), y + (height / 2)) for x, y in raw_to_points])

M = cv2.getPerspectiveTransform(from_points, to_points)

if (van_y > 0):
    offset = 75
    input_corners = np.array([[(0, height), (0, van_y + offset), (width, van_y + offset), (width, height)]], dtype=float)
else:
    input_corners = np.array([[(0, height), (0,  0), (width,  0), (width, height)]], dtype=float)
output_corners = cv2.perspectiveTransform(input_corners, M)
print(output_corners)


x_min = functools.reduce(lambda x,y: [min(x[0], y[0]), 0], output_corners[0])[0]
x_max = functools.reduce(lambda x,y: [max(x[0], y[0]), 0], output_corners[0])[0]
y_min = functools.reduce(lambda x,y: [0, min(x[1], y[1])], output_corners[0])[1]
y_max = functools.reduce(lambda x,y: [0, max(x[1], y[1])], output_corners[0])[1]

br_x = x_min
br_y = y_min

new_width = int(x_max - x_min)
new_height = int(y_max - y_min)

print(new_width, new_height)

#to_points = np.float32([(to_point[0] - 4 * br_x, to_point[1] - 15 * br_y) for to_point in to_points])
to_points = np.float32([(to_point[0] - br_x, to_point[1] - br_y) for to_point in to_points])
M_shifted = cv2.getPerspectiveTransform(from_points, to_points)

dst = cv2.warpPerspective(im, M_shifted, (new_width, new_height))
#dst = cv2.warpPerspective(im, M_shifted, (40000, 40000))
cv2.imwrite('./My_transformed.jpg', dst)

In [ ]:
sin_t, cos_t, tg_t

In [ ]:
t = from_points
A = np.array([
             [t[0][0], t[0][1], 0,       0,       -t[0][0]**2,      -t[0][0]*t[0][1]],
             [t[1][0], t[1][1], 0,       0,       -t[0][0]*t[1][0], -t[0][0]*t[1][1]],
             [t[2][0], t[1][1], 0,       0,       -t[2][0]*t[3][0], -t[1][1]*t[3][0]],
             [t[3][0], t[0][1], 0,       0,       -t[3][0]**2,      -t[0][1]*t[3][0]],
             [0,       0,       t[0][0], t[0][1], -t[0][0]*t[0][1], -t[0][1]**2     ],
             [0,       0,       t[3][0], t[0][1], -t[3][0]*t[0][1], -t[0][1]**2     ]
            ])
b = np.array([[t[0][0]], [t[0][0]], [t[3][0]], [t[3][0]], [t[0][1]], [t[0][1]]])

x0 = np.dot (np.linalg.inv(A), b).T[0]
print (x0)

M = np.array([[x0[0], x0[1], 0],
               [x0[2], x0[3], 0],
               [x0[4], x0[5], 1]
              ])

In [ ]:
#M = cv2.getPerspectiveTransform(from_points, to_points)
input_corners = np.array([[(0, height), (0, plus_up[1]), (width, minus_up[1]), (width, height)]], dtype=float)

output_corners = cv2.perspectiveTransform(input_corners, M)
print (output_corners)
x_min = functools.reduce(lambda x,y: [min(x[0], y[0]), 0], output_corners[0])[0]
x_max = functools.reduce(lambda x,y: [max(x[0], y[0]), 0], output_corners[0])[0]
y_min = functools.reduce(lambda x,y: [0, min(x[1], y[1])], output_corners[0])[1]
y_max = functools.reduce(lambda x,y: [0, max(x[1], y[1])], output_corners[0])[1]

br_x = x_min
br_y = y_min

shift = np.array ([[1, 0, -br_x],
                   [0, 1, -br_y],
                   [0, 0, 1]])
print (shift)

new_width = int(x_max - x_min)
new_height = int(y_max - y_min)

#to_points = np.float32([(to_point[0] - br_x, to_point[1] - br_y) for to_point in to_points])
M_shifted = np.dot (shift, M)

#dst = cv2.warpPerspective(im, M_shifted, (max(init_width, width), height))
dst = cv2.warpPerspective(im, M_shifted,  (new_width, new_height))
print (dst.shape)
cv2.imwrite(base+'transformed1.jpg', dst)

In [ ]:
help(cv2.boundingRect)
cv2.boundi

In [ ]:
x_min, x_max

In [ ]:
x1

In [ ]:
def get_vanishing_point(rho_1, theta_1, rho_2, theta_2):
    k_1 = - np.cos(theta_1) / np.sin(theta_1)
    m_1 = rho_1 / np.sin(theta_1)
    
    k_2 = - np.cos(theta_2) / np.sin(theta_2)
    m_2 = rho_2 / np.sin(theta_2)
    
    x = (m_1 - m_2) / (k_1 - k_2)
    y = (k_1 * m_2 - k_2 * m_1) / (k_1 - k_2)
    
    return (x, y)
